<a href="https://colab.research.google.com/github/Hitika-Jain/Article-Analysis/blob/main/article_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4 selenium pandas openpyxl nltk textstat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00


In [ ]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Load input file
input_path = "/content/Input.xlsx"
input_df = pd.read_excel(input_path)

# Ensure output directory exists
output_dir = "extracted_articles"
os.makedirs(output_dir, exist_ok=True)

def extract_article(url):
    """Extracts title, subheadings, list items, and article text from the given URL, filtering out non-article content."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url}")
            return None, None

        soup = BeautifulSoup(response.content, "html.parser")

        # Extract title (modify selector based on website structure)
        title = soup.find("h1").get_text(strip=True) if soup.find("h1") else "No Title"

        # Extract article content
        article = soup.find(class_=["td-post-content", "article-content", "post-content", "entry-content", "content-area"])

        if article:
            # Remove unwanted elements (ads, footers, social media links, etc.)
            for unwanted in article.find_all(["div", "span", "aside", "nav"], class_=lambda x: x and ("ads" in x or "footer" in x or "related" in x)):
                unwanted.decompose()

            # Extract subheadings (h1, h2, h3), paragraphs, and list items
            content_elements = article.find_all(["h1", "h2", "h3", "p", "li"])
            content = "\n".join([elem.get_text(strip=True) for elem in content_elements])
        else:
            content = ""

        return title, content
    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None, None

# Process each URL
for index, row in input_df.iterrows():
    url_id = str(row["URL_ID"])  # Ensure it's a string for file naming
    url = row["URL"]

    title, article_text = extract_article(url)

    if article_text:
        with open(os.path.join(output_dir, f"{url_id}.txt"), "w", encoding="utf-8") as f:
            f.write(f"{title}\n{article_text}")
    else:
        print(f"Skipping {url_id}: No content extracted.")

print("Extraction complete. Files saved in 'extracted_articles' directory.")


Extraction complete. Files saved in 'extracted_articles' directory.


In [ ]:
!pip install nltk textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.0 MB/s eta 0:00:00


In [ ]:
import os
print("Available files:", os.listdir("/content/"))

Available files: ['.config', 'StopWords_DatesandNumbers.txt', 'Output Data Structure.xlsx', 'StopWords_Currencies.txt', 'Input.xlsx', 'negative-words.txt', 'StopWords_Generic.txt', 'extracted_articles1', 'StopWords_GenericLong.txt', 'extracted_articles', 'StopWords_Auditor.txt', 'StopWords_Geographic.txt', 'positive-words.txt', 'StopWords_Names.txt', 'extracted_articles2', 'sample_data']


In [ ]:
import os
import pandas as pd
import nltk
import textstat
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import chardet
nltk.download("punkt")
nltk.download('punkt_tab')

nltk.download("stopwords")

# Load stop words
stopword_files = ["/content/StopWords_Auditor.txt", "/content/StopWords_Generic.txt", "/content/StopWords_GenericLong.txt",
                  "/content/StopWords_Currencies.txt", "/content/StopWords_DatesandNumbers.txt", "/content/StopWords_Geographic.txt", "/content/StopWords-Names.txt"]
stop_words = set()
for file in stopword_files:
    print(f"Processing file: {file}")  # To verify that each file is being accessed
    try:
        with open(f"{file}", "r", encoding="ISO-8859-1") as f:
            stop_words.update(f.read().splitlines())
    except FileNotFoundError:
        print(f"File not found: {file}")
    except Exception as e:
        print(f"Error processing file {file}: {e}")

print(f"Total stop words: {len(stop_words)}")

# Load positive and negative words
with open("/content/positive-words.txt", "r", encoding="utf-8") as f:
    positive_words = set(f.read().splitlines())

with open("/content/negative-words.txt", "r", encoding="ISO-8859-1") as f:
    negative_words = set(f.read().splitlines())


def analyze_text(text):
    words = [word.lower() for word in word_tokenize(text) if word.isalpha()]  # Keep only alphabetic words
    sentences = sent_tokenize(text)

    # Debugging: Check tokenized words
    print(f"Tokenized words: {words[:20]}...")  # Print first 20 tokens for review

    # Sentiment Analysis
    positive_score = len([word for word in words if word in positive_words])
    negative_score = len([word for word in words if word in negative_words])

    # Debugging: Check positive and negative matches
    print(f"Positive words found: {[word for word in words if word in positive_words]}")
    print(f"Negative words found: {[word for word in words if word in negative_words]}")
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)

    # Readability Metrics
    avg_sentence_length = len(words) / max(len(sentences), 1)
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    percentage_complex_words = len(complex_words) / max(len(words), 1) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Additional Metrics
    word_count = len([word for word in words if word.lower() not in stop_words])
    avg_word_count_per_sentence = len(words) / max(len(sentences), 1)
    syllable_per_word = sum(textstat.syllable_count(word) for word in words) / max(len(words), 1)
    avg_word_length = sum(len(word) for word in words) / max(len(words), 1)
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

    return [positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_complex_words, fog_index,avg_word_count_per_sentence,
            len(complex_words), word_count, syllable_per_word, personal_pronouns, avg_word_length]

# Load output structure
output_path = "/content/Output Data Structure.xlsx"
output_df = pd.read_excel(output_path)
output_columns = output_df.columns.tolist()
url_id_column = output_df.columns[0]  # First column (URL ID)
link_column = output_df.columns[1]    # Second column (link)
# Process extracted articles
article_dir = "extracted_articles2"
output_data = []

for file in os.listdir(article_dir):
    with open(os.path.join(article_dir, file), "r", encoding="utf-8") as f:
        text = f.read()

    url_id = file.replace(".txt", "")  # URL ID (file name)
    # Get the corresponding link from the input file based on the URL ID
    link = input_df[input_df[url_id_column] == url_id][link_column].values
    if len(link) > 0:
        link = link[0]  # Extract the first match (assuming unique URL IDs)
    else:
        link = "No link found"  # In case no match is found

    # Get the analysis result (13 values)
    analysis_result = analyze_text(text)

    # Append the URL ID, link, and analysis results (13 values starting from column 3)
    output_data.append([url_id, link] + analysis_result)

# Verify that output_data has the correct shape
print(f"Total rows in output_data: {len(output_data)}")
print(f"Total columns in output_df: {output_df.shape[1]}")

# Add the new data to the DataFrame (filling starting from column C)
for i, row in enumerate(output_data):
    if len(row[2:]) == (output_df.shape[1] - 2):  # Ensure the number of values to fill matches the columns
        output_df.iloc[i, 2:] = row[2:]  # Fill from column C (index 2) onwards
    else:
        print(f"Skipping row {i} due to mismatch in length: {len(row[2:])} vs {output_df.shape[1] - 2}")

# Save the modified DataFrame back to Excel
output_df.to_excel("Final_Output.xlsx", index=False)

print("Text analysis completed. Results saved in 'Final_Output.xlsx'")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing file: /content/StopWords_Auditor.txt
Processing file: /content/StopWords_Generic.txt
Processing file: /content/StopWords_GenericLong.txt
Processing file: /content/StopWords_Currencies.txt
Processing file: /content/StopWords_DatesandNumbers.txt
Processing file: /content/StopWords_Geographic.txt
Processing file: /content/StopWords-Names.txt
File not found: /content/StopWords-Names.txt
Total stop words: 1087
Tokenized words: ['quickbooks', 'dashboard', 'to', 'find', 'patterns', 'in', 'finance', 'sales', 'and', 'forecasts', 'client', 'background', 'client', 'a', 'leading', 'marketing', 'firm', 'in', 'the', 'usa']...
Positive words found: ['leading', 'integrated', 'integrated', 'friendly', 'useful', 'integrated', 'available', 'successfully', 'refresh', 'refresh', 'refresh', 'like', 'top', 'top', 'refresh', 'refresh', 'work', 'dynamic', 'refresh', 'dynamic', 'dynamic', 'refresh', 'refresh', 'refresh', 'proper', 'dynamic', 'refresh', 'dynamic', 'refresh', 'refresh', 'refresh']
Nega

In [ ]:
import re
from nltk.tokenize import word_tokenize

# Choose the file you want to check (replace this with the path to your file)
file_path = "/content/extracted_articles2/Netclan20241017.txt"

# Read the file's content
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Tokenize the words in the text
# Tokenize the words
words = word_tokenize(text)

# Debug: Show the first 20 tokenized words
print(f"Tokenized words: {words[:20]}")

# Check for positive and negative words in the tokenized words
positive_matches = [word for word in words if word.lower() in positive_words]
negative_matches = [word for word in words if word.lower() in negative_words]

# Debug: Show matched positive and negative words
print(f"Positive words found: {positive_matches}")
print(f"Negative words found: {negative_matches}")

# Calculate positive and negative scores
positive_score = len(positive_matches)
negative_score = len(negative_matches)

# Print the final scores
print(f"Positive score: {positive_score}")
print(f"Negative score: {negative_score}")

# Optionally: Also verify that the stopwords are being correctly excluded
words_without_stopwords = [word for word in words if word.lower() not in stop_words]
print(f"Words without stopwords: {words_without_stopwords[:20]}")



Tokenized words: ['AI', 'and', 'ML-Based', 'YouTube', 'Analytics', 'and', 'Content', 'Creation', 'Tool', 'for', 'Optimizing', 'Subscriber', 'Engagement', 'and', 'Content', 'Strategy', 'Client', 'Background', 'Client', ':']
Positive words found: ['leading', 'Support', 'integrated', 'Supports', 'Enhancement']
Negative words found: ['Problem', 'Cloud', 'Cloud']
Positive score: 5
Negative score: 3
Words without stopwords: ['AI', 'ML-Based', 'YouTube', 'Analytics', 'Content', 'Creation', 'Tool', 'Optimizing', 'Subscriber', 'Engagement', 'Content', 'Strategy', 'Client', 'Background', 'Client', ':', 'leading', '&', 'tech', 'firm']


In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import nltk
import textstat
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("stopwords")

# Load stop words
stopword_files = [
    "/content/StopWords_Auditor.txt",
    "/content/StopWords_Generic.txt",
    "/content/StopWords_GenericLong.txt",
    "/content/StopWords_Currencies.txt",
    "/content/StopWords_DatesandNumbers.txt",
    "/content/StopWords_Geographic.txt",
    "/content/StopWords-Names.txt",
]

stop_words = set()
for file in stopword_files:
    try:
        with open(file, "r", encoding="ISO-8859-1") as f:
            stop_words.update(f.read().splitlines())
    except FileNotFoundError:
        print(f"File not found: {file}")

# Load positive and negative words
with open("/content/positive-words.txt", "r", encoding="utf-8") as f:
    positive_words = set(f.read().splitlines())

with open("/content/negative-words.txt", "r", encoding="ISO-8859-1") as f:
    negative_words = set(f.read().splitlines())

# Read the input file (contains URL ID, Link)
input_path = "/content/Input.xlsx"
input_df = pd.read_excel(input_path)

# Read the output structure file
output_path = "/content/Output Data Structure.xlsx"
output_df = pd.read_excel(output_path)

# Function to analyze text
def analyze_text(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)

    # Sentiment Analysis
    positive_score = len([word for word in words if word.lower() in positive_words])
    negative_score = len([word for word in words if word.lower() in negative_words])
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)

    # Readability Metrics
    avg_sentence_length = len(words) / max(len(sentences), 1)
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    percentage_complex_words = len(complex_words) / max(len(words), 1) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Additional Metrics
    word_count = len([word for word in words if word.lower() not in stop_words])
    avg_word_count_per_sentence = len(words) / max(len(sentences), 1)
    syllable_per_word = sum(textstat.syllable_count(word) for word in words) / max(len(words), 1)
    avg_word_length = sum(len(word) for word in words) / max(len(words), 1)
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

    return [
        positive_score, negative_score, polarity_score, subjectivity_score,
        avg_sentence_length, percentage_complex_words, fog_index, avg_word_count_per_sentence,
        len(complex_words), word_count, syllable_per_word, personal_pronouns, avg_word_length
    ]

# Directory where extracted articles are stored
article_dir = "/content/extracted_articles2"

# Create a dictionary to store processed results
processed_data = {}

# Process each article
for file in os.listdir(article_dir):
    url_id = file.replace(".txt", "")  # Extract URL ID from filename

    # Read the text content
    with open(os.path.join(article_dir, file), "r", encoding="utf-8") as f:
        text = f.read()

    # Find the corresponding link
    link_row = input_df[input_df.iloc[:, 0] == url_id]
    if not link_row.empty:
        link = link_row.iloc[0, 1]  # Assuming link is in second column
    else:
        link = "No link found"

    # Analyze text and store the results
    analysis_result = analyze_text(text)
    processed_data[url_id] = [link] + analysis_result  # Store link & metrics

# Ensure output DataFrame is correctly populated
for index, row in output_df.iterrows():
    url_id = str(row.iloc[0])  # Extract URL ID from the output structure

    if url_id in processed_data:
        # Fill corresponding row starting from column C (index 2)
        output_df.iloc[index, 1:] = processed_data[url_id]

# Save final output
output_df.to_excel("Final_Output.xlsx", index=False)

print("Text analysis completed. Results saved in 'Final_Output.xlsx'.")

File not found: /content/StopWords-Names.txt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Text analysis completed. Results saved in 'Final_Output.xlsx'.


In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
import pandas as pd
import nltk
import textstat
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("stopwords")

# Load stop words
stopword_files = [
    "/content/StopWords_Auditor.txt",
    "/content/StopWords_Generic.txt",
    "/content/StopWords_GenericLong.txt",
    "/content/StopWords_Currencies.txt",
    "/content/StopWords_DatesandNumbers.txt",
    "/content/StopWords_Geographic.txt",
    "/content/StopWords-Names.txt",
]

stop_words = set()
for file in stopword_files:
    try:
        with open(file, "r", encoding="ISO-8859-1") as f:
            stop_words.update(f.read().splitlines())
    except FileNotFoundError:
        print(f"File not found: {file}")

# Load positive and negative words
with open("/content/positive-words.txt", "r", encoding="utf-8") as f:
    positive_words = set(f.read().splitlines())

with open("/content/negative-words.txt", "r", encoding="ISO-8859-1") as f:
    negative_words = set(f.read().splitlines())

# Read input file
input_path = "/content/Input.xlsx"
input_df = pd.read_excel(input_path)

# Ensure output directory exists
output_dir = "extracted_articles"
os.makedirs(output_dir, exist_ok=True)

def extract_article(url):
    """Extracts title, subheadings, list items, and article text from the given URL."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url}")
            return None, None

        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find("h1").get_text(strip=True) if soup.find("h1") else "No Title"
        article = soup.find(class_=["td-post-content", "article-content", "post-content", "entry-content", "content-area"])

        if article:
            for unwanted in article.find_all(["div", "span", "aside", "nav"], class_=lambda x: x and ("ads" in x or "footer" in x or "related" in x)):
                unwanted.decompose()
            content_elements = article.find_all(["h1", "h2", "h3", "p", "li"])
            content = "\n".join([elem.get_text(strip=True) for elem in content_elements])
        else:
            content = ""

        return title, content
    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None, None

# Extract articles
for index, row in input_df.iterrows():
    url_id = str(row["URL_ID"])
    url = row["URL"]
    title, article_text = extract_article(url)

    if article_text:
        with open(os.path.join(output_dir, f"{url_id}.txt"), "w", encoding="utf-8") as f:
            f.write(f"{title}\n{article_text}")
    else:
        print(f"Skipping {url_id}: No content extracted.")

print("Extraction complete. Files saved in 'extracted_articles' directory.")

# Read output structure file
output_path = "/content/Output Data Structure.xlsx"
output_df = pd.read_excel(output_path)

def analyze_text(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    positive_score = len([word for word in words if word.lower() in positive_words])
    negative_score = len([word for word in words if word.lower() in negative_words])
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)
    avg_sentence_length = len(words) / max(len(sentences), 1)
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    percentage_complex_words = len(complex_words) / max(len(words), 1) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    word_count = len([word for word in words if word.lower() not in stop_words])
    avg_word_count_per_sentence = len(words) / max(len(sentences), 1)
    syllable_per_word = sum(textstat.syllable_count(word) for word in words) / max(len(words), 1)
    avg_word_length = sum(len(word) for word in words) / max(len(words), 1)
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

    return [
        positive_score, negative_score, polarity_score, subjectivity_score,
        avg_sentence_length, percentage_complex_words, fog_index, avg_word_count_per_sentence,
        len(complex_words), word_count, syllable_per_word, personal_pronouns, avg_word_length
    ]

# Analyze extracted articles
processed_data = {}
for file in os.listdir(output_dir):
    url_id = file.replace(".txt", "")
    with open(os.path.join(output_dir, file), "r", encoding="utf-8") as f:
        text = f.read()
    link_row = input_df[input_df.iloc[:, 0] == url_id]
    link = link_row.iloc[0, 1] if not link_row.empty else "No link found"
    analysis_result = analyze_text(text)
    processed_data[url_id] = [link] + analysis_result

# Fill output DataFrame
for index, row in output_df.iterrows():
    url_id = str(row.iloc[0])
    if url_id in processed_data:
        output_df.iloc[index, 1:] = processed_data[url_id]

#final output
output_df.to_excel("Final_Output.xlsx", index=False)
print("Text analysis completed. Results saved in 'Final_Output.xlsx'.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


File not found: /content/StopWords-Names.txt
Extraction complete. Files saved in 'extracted_articles' directory.
Text analysis completed. Results saved in 'Final_Output.xlsx'.


In [ ]:
!ls /content/

 negative-words.txt	       StopWords_Auditor.txt	       StopWords_Generic.txt
'Output Data Structure.xlsx'   StopWords_Currencies.txt        StopWords_Geographic.txt
 positive-words.txt	       StopWords_DatesandNumbers.txt   StopWords_Names.txt
 sample_data		       StopWords_GenericLong.txt
